# Import Required Modules

In [1]:
import torch
import os
from model.LLMModel import LLMModel
from model.LLMLayer import LLMLayer
from utils.tokenizer import get_tokenizer
from utils.training_data_generator import TrainingDataGen
from utils.trainer import Trainer
from utils.read_config import get_config

In [2]:
config = get_config()

# Create Tokenizer object

In [3]:
tokenizer = get_tokenizer()

# Initialize Model

In [4]:

embedding_dimension = 512
max_sequence_length = 2048
number_of_tokens = tokenizer.get_vocab_size()
batch_size = 1
data_gen_config_path = config['checkpoint_path']
checkpoint_files = os.listdir(config['checkpoint_path'])
data_gen_config_path = config['checkpoint_path']
model_checkpoint_path = None
for file in checkpoint_files:
    if file.find(".pt") > -1:
        model_checkpoint_path = config['checkpoint_path']+file

# Create the model
model = LLMModel(LLMLayer(
    embedding_dimension=embedding_dimension,
    number_of_tokens=number_of_tokens,
    number_of_heads=4,
    number_of_layers=64,
    decoder_stacks=2,
    dropout_rate=0.1,
    max_sequence_length=max_sequence_length,
    feed_forward_dimension=4*embedding_dimension,
    training = True
))

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)


# Initialize the Data Generator

In [5]:
if data_gen_config_path:
    if os.path.exists(os.path.join(data_gen_config_path,config['data_genrerator_config_name'])):
        training_data_generator = TrainingDataGen(config_path=data_gen_config_path)
    else:
        training_data_generator = TrainingDataGen(base_path=config['data_path'])
else:
    training_data_generator = TrainingDataGen(base_path=config['data_path'])

# Calculate Model Parameters

In [6]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(int(trainable_params)/pow(10,9))

0.48941816


# Start Training

In [7]:
trainer = Trainer(model, tokenizer, optimizer, max_sequence_length, model_checkpoint_path, batch_size)
trainer.train(training_data_generator, epochs=100, batch_size=batch_size)

  0%|          | 0/100 [26:41<?, ?it/s]

Current File:  d:/Data/books/0.txt
Step 39 Input Tensor: [' Xander. As always, tremendous thanks also goes to the rest of my family: my parents, Tamara and Sasha; my sister, Liz, and her wonderful husband, Justin, and their kids, Hannah and Max; my brother, AJ, and his wife, Ali; my grandma Esther, my fantastic brother- in- law, Alex, and his wife, Sandy, and their kids, Zach and Dylan; and my mother- and father- in- law, Anne and Peter Klein. A huge thank- you to my team, aka the Dope Den: Steve Unwin, Zak Moy, David Rock, India Kieser, Alex De Simone, Andy Krainak, Andrew Greif, Staphon Lawrence, Brittany Hoffman, Siddarth Astir, and Rebecca Wright. My gratitude also goes out to the whole team at HarperBusiness. And especially to Hollis Heimbouch, who says this is the best book I\'ve written. Thank you for your consistent support and allowing me to roll the way I need to. Finally, thanks again to my writer, Stephanie Land, who beside my brother and dad has been my closest partner in 